#RAG with Langchain using Flor 1.3B (finetuned)

FLOR-1.3B Customer Service Chatbot (Catalan) with RAG and Langchain
This Colab presents an advanced multilingual support configuration using the FLOR-1.3B-customerservice language model, specifically adjusted for a customer service chatbot in Catalan. This model is an adaptation of the projecte-aina/FLOR-1.3B, first finetuned to be more instructed as a Customer Service chatbot and later optimized with Retrieval-Augmented Generation (RAG) techniques and Langchain. It takes advantage of the transformer-based architecture of FLOR-1.3B-customerservice for text generation in Catalan and enriches it with external knowledge sources to achieve context-aware results. Key components include dependency installation, efficient model loading, conversational prompts, text retrieval embeddings, and a customized chatbot named Firulais, designed to provide specific company support. This setup demonstrates a seamless integration of language modeling, retrieval, and conversational AI.

Retrieval-Augmented Generation (RAG) is a method that combines the power of transformer-based language models with the ability to query external databases to enhance the quality and factual accuracy of the generated text. Langchain is a framework designed to facilitate the integration of language models with external knowledge sources, such as databases and APIs, allowing for more informed and context-aware text generation. FLOR-1.3B-customerservice, a derivative of BLOOM, is a multi-lingual, transformer-based causal language model specifically tuned for Catalan, Spanish, and English. It was developed by adapting the vocabulary and embedding layer of the original model and further pre-training it on a massive corpus of 140B tokens in the target languages. Although primarily intended for causal language modeling, FLOR-1.3B's architecture allows it to be readily applied to text generation tasks and can be further fine-tuned for specific applications. The combination of RAG with Langchain and leveraging the capabilities of FLOR-1.3B provides a robust framework for generating contextually rich and accurate language outputs.

---
---


- More info about RAG: [Link](https://ai.meta.com/blog/retrieval-augmented-generation-streamlining-the-creation-of-intelligent-natural-language-processing-models/)

- More info about the Framework Langchain: [Link](https://www.langchain.com/)

- More info about FLOR-1.3B-customerservice: [Link](https://huggingface.co/ericrisco/flor-1-3B-customerservice)

- More info about "Projecte AINA": [Link](https://projecteaina.cat/)


## Installing dependencies

In [ ]:
!pip install -qU bitsandbytes datasets accelerate loralib peft transformers trl langchain sentence-transformers faiss-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 65.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 810.5/810.5 kB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━

## Imports and device prep

This code sets up an environment for language models, particularly for conversation and retrieval-augmented generation. It imports libraries for model interaction, tokenization, and embedding, and establishes a pipeline for conversational models with retrieval capabilities. It also configures the computing device, preferring GPU ('cuda') if available for enhanced performance.

In [ ]:
from typing import List
import transformers
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
from transformers import BloomForCausalLM, BloomTokenizerFast
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))

Device: cuda
Tesla T4


In [ ]:
import torch
torch.cuda.is_available()

True

## Initializing FLOR-1.3B-Customerservice: Efficient Model Loading and Tokenization

The code initializes and loads the FLOR-1.3B-Customerservice

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id  = "ericrisco/flor-1-3B-customerservice"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/552 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/844k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/503k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/140 [00:00<?, ?B/s]

## Setting Up a Text Generation Pipeline with Hugging Face's Transformers Library

The code snippet initializes a text generation pipeline using the Hugging Face transformers library. It sets up a model with specific tokenizer, end-of-sequence, and padding tokens, applies a repetition penalty to discourage repeated text, and limits output length. The initialized pipeline is wrapped into a HuggingFacePipeline object, flor_llm, for further use.

In [ ]:
import transformers
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=100,
)
flor_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

## Language Query with FLOR LLM

The code sends a question, "Quina és la capital d'Andorra?" (What is the capital of Andorra?), to the FLOR language model (LLM). The model processes the text and provides an answer, leveraging its pre-trained knowledge about language and factual information.

In [ ]:
text = "Quina és la capital d'Andorra?"
flor_llm.invoke(text)

"Quina és la capital d'Andorra?\n\n### Context\nLa capital d'Andorra és Andorra la Vella. La ciutat té una població de 7.660 habitants i està situada a 1.090 metres sobre el nivell del mar. La ciutat es troba al centre del país, a la vall de la Ricarda, que forma part de la vall de l'Estiula. La ciutat està connectada amb la resta del país per un sistema de carreteres i ferrocarrils.\n"

## Language Query with Flor and Langchain

This code uses Langchain's PromptTemplate to generate context-specific jokes. It creates a template for jokes, formats it with given adjectives and contexts, and then uses the FLOR language model to generate a joke based on the provided context of "informàtica" (computing) with a "penós" (cringe-worthy) theme.

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "Quin és l'import de la meva factura {bill_number}"
)
prompt.format(bill_number="5648")

llm_chain = prompt | flor_llm
llm_chain.invoke({"bill_number": "5648"})

"Quin és l'import de la meva factura 5648?\n\n### Context\nSi teniu problemes per accedir al vostre compte, podeu contactar amb el nostre equip d'atenció al client a través del telèfon gratuït 900 822 960 o mitjançant l'aplicació Account Navigator. Us guiarem a través del procés de recuperació de la contrasenya i us proporcionarem informació sobre els passos necessaris per restablir el vostre compte. Si necessiteu assistència addicional per recuperar la vostra contrasenya, podeu contactar amb el nostre equip d'assistència al client a"

Same but with Console Callback

In [ ]:
llm_chain = prompt | flor_llm
llm_chain.invoke({"bill_number": "5648"},
                 config={'callbacks': [ConsoleCallbackHandler()]})

[chain/start] [1:chain:RunnableSequence] Entering Chain run with input:
{
  "bill_number": "5648"
}
[chain/start] [1:chain:RunnableSequence > 2:prompt:PromptTemplate] Entering Prompt run with input:
{
  "bill_number": "5648"
}
[chain/end] [1:chain:RunnableSequence > 2:prompt:PromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [1:chain:RunnableSequence > 3:llm:HuggingFacePipeline] Entering LLM run with input:
{
  "prompts": [
    "Quin és l'import de la meva factura 5648"
  ]
}
[llm/end] [1:chain:RunnableSequence > 3:llm:HuggingFacePipeline] [3.54s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "Quin és l'import de la meva factura 5648?\n\n### Context\nSi teniu problemes per accedir al vostre compte, podeu contactar amb el nostre equip d'atenció al client a través del telèfon gratuït 900 822 960 o mitjançant l'aplicació Account Navigator. Us guiarem a través del procés de recuperació de la contrasenya i us proporcionarem informació 

"Quin és l'import de la meva factura 5648?\n\n### Context\nSi teniu problemes per accedir al vostre compte, podeu contactar amb el nostre equip d'atenció al client a través del telèfon gratuït 900 822 960 o mitjançant l'aplicació Account Navigator. Us guiarem a través del procés de recuperació de la contrasenya i us proporcionarem informació sobre els passos necessaris per restablir el vostre compte. Si necessiteu assistència addicional per recuperar la vostra contrasenya, podeu contactar amb el nostre equip d'assistència al client a"

## Creation and Execution of a Custom Chat Assistant

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful Assistant. Your name is Firulais. Answer with short sentences and always in catalan."),
    ]
)

llm_chain = chat_prompt | flor_llm
llm_chain.invoke({"user_input": "What is your name?"})

'System: You are a helpful Assistant. Your name is Firulais. Answer with short sentences and always in catalan.\n\n### Answer\nFirulais : Assistente, catalan : idioma'

Really bad answer XD

## Efficient Text Embeddings with HuggingFace MiniLM

This code initializes the HuggingFaceEmbeddings class from the Langchain library, utilizing the compact and efficient 'all-MiniLM-l6-v2' model from sentence-transformers for generating text embeddings, and sets the model to run on a CUDA device for GPU acceleration.

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",
    model_kwargs={"device": "cuda"},
)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Internal Document Integration and Insights

The code represents a simplified process of loading internal company documents from repositories such as SharePoint into a variable named db_docs_gti. Although the actual loading process from sources is complex and crucial, the example bypasses this step for simplicity. The documents, already loaded, contain detailed insights on Global Tech Innovations (GTI), covering aspects like company overview, telework, environmental, diversity, and innovation policies. This enables streamlined access and analysis of vital company information.

In [ ]:
db_docs_gti = [
  "Gamma de Productes de Global Tech Innovations (GTI): GTI, líder en el sector tecnològic, ofereix una àmplia gamma de productes, incloent eines avançades d'IA i aprenentatge automàtic. Coneguda per prioritzar la sostenibilitat i la innovació, els productes de GTI estan dissenyats amb tecnologia d'avantguarda i pràctiques respectuoses amb el medi ambient. La presència global de l'empresa es reflecteix en les seves diverses ofertes de productes, satisfent les necessitats de diversos mercats i promovent una cultura d'excel·lència tecnològica.",
  "Política de Suport de Productes de GTI: GTI proporciona un suport integral pels seus productes, assegurant que els clients rebin assistència ràpida i eficient. La política inclou un equip dedicat d'atenció al client, disponible per a suport remot i solució de problemes. GTI es compromet a oferir un suport de primera, garantint la fiabilitat del producte i la satisfacció del client.",
  "Estàndards Ambientals de GTI pels Productes: El compromís de GTI amb el medi ambient s'estén al disseny i fabricació dels seus productes. L'empresa implementa pràctiques ecològiques, com l'ús de materials sostenibles i la reducció d'emissions de carboni durant la producció. Els productes de GTI estan dissenyats per ser energèticament eficients i respectuosos amb el medi ambient, alineant-se amb els objectius de sostenibilitat de l'empresa.",
  "Diversitat i Accessibilitat dels Productes de GTI: GTI està dedicada a crear productes accessibles i útils per a una àmplia gamma de clients. L'empresa es centra en el disseny inclusiu, assegurant que els productes satisfacin diferents habilitats i necessitats. El compromís de GTI amb la diversitat és evident en les seves estratègies de desenvolupament de productes, amb l'objectiu de proporcionar tecnologia que millori la vida de tots els usuaris.",
  "Innovació en el Desenvolupament de Productes de GTI: La innovació és la pedra angular del desenvolupament de productes de GTI. L'empresa inverteix molt en recerca i desenvolupament per crear productes d'avantguarda que ampliïn els límits de la tecnologia. GTI fomenta una cultura d'innovació, donant suport a iniciatives que condueixin a dissenys i funcionalitats de productes nous i millorats."
]

## Implementing Text Retrieval with FAISS and Langchain

This code initializes a FAISS-based vector database using document embeddings and creates a retriever object. It converts textual data into a vector space for efficient similarity searches, enabling rapid retrieval of relevant documents from the database based on vector proximity.

In [ ]:
from langchain.vectorstores import FAISS
from langchain_core.vectorstores import VectorStoreRetriever

vector_db = FAISS.from_texts(db_docs_gti, embeddings)
retriever = VectorStoreRetriever(vectorstore=vector_db)

## Implementation of a Multilingual Support Chatbot

This code defines a template for a multilingual support chatbot named Firulais, specializing in company-related inquiries. It sets up a question-answering system that utilizes the FLOR language model, with structured guidelines ensuring responses are in Catalan, accurate, and within the bot's knowledge domain. The system handles conversation history and corrects spelling errors, enhancing user interaction.

In [ ]:
system_message = "You are a helpfull customer service assistant, your name is Firulais. Answer always in catalan."

template = "<|startoftext|>"
template += "\n### Instruction\n"
template += system_message + "\n\n"
template += "### Context\n "
template += "{context}\n "
template += "History = {history}\n "
template += "Question = {question}\n "
template += "### Answer\n"

prompt = PromptTemplate(
        template=template, input_variables=["history", "context", "question"]
    )
qa = RetrievalQA.from_chain_type(
        llm=flor_llm,
        chain_type="stuff",
        retriever=retriever,
        chain_type_kwargs={
            "verbose": False,
            "prompt": prompt,
            "memory": ConversationBufferMemory(
                memory_key="history",
                input_key="question"),
        }
    )

In [ ]:
qa.run("Com et dius?")

"<|startoftext|>\n### Instruction\nYou are a helpfull customer service assistant, your name is Firulais. Answer always in catalan.\n\n### Context\n Diversitat i Accessibilitat dels Productes de GTI: GTI està dedicada a crear productes accessibles i útils per a una àmplia gamma de clients. L'empresa es centra en el disseny inclusiu, assegurant que els productes satisfacin diferents habilitats i necessitats. El compromís de GTI amb la diversitat és evident en les seves estratègies de desenvolupament de productes, amb l'objectiu de proporcionar tecnologia que millori la vida de tots els usuaris.\n\nGamma de Productes de Global Tech Innovations (GTI): GTI, líder en el sector tecnològic, ofereix una àmplia gamma de productes, incloent eines avançades d'IA i aprenentatge automàtic. Coneguda per prioritzar la sostenibilitat i la innovació, els productes de GTI estan dissenyats amb tecnologia d'avantguarda i pràctiques respectuoses amb el medi ambient. La presència global de l'empresa es refle

In [ ]:
qa.run("Quins son els productes principals?")

"<|startoftext|>\n### Instruction\nYou are a helpfull customer service assistant, your name is Firulais. Answer always in catalan.\n\n### Context\n Política de Suport de Productes de GTI: GTI proporciona un suport integral pels seus productes, assegurant que els clients rebin assistència ràpida i eficient. La política inclou un equip dedicat d'atenció al client, disponible per a suport remot i solució de problemes. GTI es compromet a oferir un suport de primera, garantint la fiabilitat del producte i la satisfacció del client.\n\nInnovació en el Desenvolupament de Productes de GTI: La innovació és la pedra angular del desenvolupament de productes de GTI. L'empresa inverteix molt en recerca i desenvolupament per crear productes d'avantguarda que ampliïn els límits de la tecnologia. GTI fomenta una cultura d'innovació, donant suport a iniciatives que condueixin a dissenys i funcionalitats de productes nous i millorats.\n\nDiversitat i Accessibilitat dels Productes de GTI: GTI està dedica

In [ ]:
qa.run("Quina estrategia implementa la companyia en temes de sostenibilitat")

"<|startoftext|>\n### Instruction\nYou are a helpfull customer service assistant, your name is Firulais. Answer always in catalan.\n\n### Context\n Gamma de Productes de Global Tech Innovations (GTI): GTI, líder en el sector tecnològic, ofereix una àmplia gamma de productes, incloent eines avançades d'IA i aprenentatge automàtic. Coneguda per prioritzar la sostenibilitat i la innovació, els productes de GTI estan dissenyats amb tecnologia d'avantguarda i pràctiques respectuoses amb el medi ambient. La presència global de l'empresa es reflecteix en les seves diverses ofertes de productes, satisfent les necessitats de diversos mercats i promovent una cultura d'excel·lència tecnològica.\n\nEstàndards Ambientals de GTI pels Productes: El compromís de GTI amb el medi ambient s'estén al disseny i fabricació dels seus productes. L'empresa implementa pràctiques ecològiques, com l'ús de materials sostenibles i la reducció d'emissions de carboni durant la producció. Els productes de GTI estan di